In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 


In [7]:
df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [11]:
df['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

#### AdaBoost Classifier 

In [12]:
y=df['Outcome']
X=df.drop('Outcome',axis=1)

In [14]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,y,train_size=0.7,random_state=42)

In [15]:
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score,roc_curve

In [16]:
adac= AdaBoostClassifier(random_state=4)

adac.fit(xtrain,ytrain)
ytrain_pred=adac.predict(xtrain)
ytrain_prob=adac.predict_proba(xtrain)[:,1]

print('Train Results \n ')
print('Confusion Matrix \n')
print(confusion_matrix(ytrain,ytrain_pred))
print('**************')
print('Accuracy SCore',accuracy_score(ytrain,ytrain_pred))
print('ROC_AUC_SCORE',roc_auc_score(ytrain,ytrain_prob))


ytest_pred=adac.predict(xtest)
ytest_prob=adac.predict_proba(xtest)[:,1]

print('\n')
print('Test Results ')
print('Confusion Matrix ')
print('*************************************')
print(confusion_matrix(ytest,ytest_pred),'\n')
print('Accuracy Score',accuracy_score(ytest,ytest_pred))
print('ROC_AUC_SCORE',roc_auc_score(ytest,ytest_prob))



Train Results 
 
Confusion Matrix 

[[319  30]
 [ 49 139]]
**************
Accuracy SCore 0.8528864059590316
ROC_AUC_SCORE 0.9367646162287386


Test Results 
Confusion Matrix 
*************************************
[[122  29]
 [ 30  50]] 

Accuracy Score 0.7445887445887446
ROC_AUC_SCORE 0.7551738410596026


### LightGBM

In [18]:
pip install LightGBM

Note: you may need to restart the kernel to use updated packages.


In [20]:
import lightgbm as lgb

In [24]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()

xtrains=ss.fit_transform(xtrain)
xtests=ss.transform(xtest)

In [25]:
lgbc= lgb.LGBMClassifier(random_state=4)



lgbc.fit(xtrains,ytrain)
ytrain_pred=lgbc.predict(xtrains)
ytrain_prob=lgbc.predict_proba(xtrains)[:,1]

print('Train Results \n ')
print('Confusion Matrix \n')
print(confusion_matrix(ytrain,ytrain_pred))
print('**************')
print('Accuracy SCore',accuracy_score(ytrain,ytrain_pred))
print('ROC_AUC_SCORE',roc_auc_score(ytrain,ytrain_prob))


ytest_pred=lgbc.predict(xtests)
ytest_prob=lgbc.predict_proba(xtests)[:,1]

print('\n')
print('Test Results ')
print('Confusion Matrix ')
print('*************************************')
print(confusion_matrix(ytest,ytest_pred),'\n')
print('Accuracy Score',accuracy_score(ytest,ytest_pred))
print('ROC_AUC_SCORE',roc_auc_score(ytest,ytest_prob))



Train Results 
 
Confusion Matrix 

[[349   0]
 [  0 188]]
**************
Accuracy SCore 1.0
ROC_AUC_SCORE 1.0


Test Results 
Confusion Matrix 
*************************************
[[116  35]
 [ 26  54]] 

Accuracy Score 0.7359307359307359
ROC_AUC_SCORE 0.7939569536423841


### HYper Parameter Tuning of LightGBM

In [31]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint 
from scipy.stats import uniform as sp_uniform

In [37]:
Xs=ss.fit_transform(X)
lgbc=lgb.LGBMClassifier(random_state=4)

params={'n_estimators':sp_randint(20,250),
       'max_depth':sp_randint(2,10),
       'learning_rate':sp_uniform(0,0.5),
       'num_leaves':sp_randint(10,100)}
rsearch=RandomizedSearchCV(lgbc,param_distributions=params,n_iter=100,cv=3,scoring='roc_auc',n_jobs=-1,random_state=4,
                           return_train_score=True)

rsearch.fit(Xs,y)


RandomizedSearchCV(cv=3, estimator=LGBMClassifier(random_state=4), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001379C2590C8>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001379C259288>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001379C229B48>,
                                        'num_leaves': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001379C243D48>},
                   random_state=4, return_train_score=True, scoring='roc_auc')

In [38]:
rsearch.best_params_

{'learning_rate': 0.1603078223495974,
 'max_depth': 3,
 'n_estimators': 23,
 'num_leaves': 84}

In [39]:
lgbc= lgb.LGBMClassifier(**rsearch.best_params_,random_state=4)



lgbc.fit(xtrains,ytrain)
ytrain_pred=lgbc.predict(xtrains)
ytrain_prob=lgbc.predict_proba(xtrains)[:,1]

print('Train Results \n ')
print('Confusion Matrix \n')
print(confusion_matrix(ytrain,ytrain_pred))
print('**************')
print('Accuracy SCore',accuracy_score(ytrain,ytrain_pred))
print('ROC_AUC_SCORE',roc_auc_score(ytrain,ytrain_prob))


ytest_pred=lgbc.predict(xtests)
ytest_prob=lgbc.predict_proba(xtests)[:,1]

print('\n')
print('Test Results ')
print('Confusion Matrix ')
print('*************************************')
print(confusion_matrix(ytest,ytest_pred),'\n')
print('Accuracy Score',accuracy_score(ytest,ytest_pred))
print('ROC_AUC_SCORE',roc_auc_score(ytest,ytest_prob))



Train Results 
 
Confusion Matrix 

[[313  36]
 [ 45 143]]
**************
Accuracy SCore 0.8491620111731844
ROC_AUC_SCORE 0.9324361397305371


Test Results 
Confusion Matrix 
*************************************
[[119  32]
 [ 28  52]] 

Accuracy Score 0.7402597402597403
ROC_AUC_SCORE 0.8116721854304636
